In [9]:
import os  # Importing the os module for interacting with the operating system

# [Student: Mohamed] Creating folder 'data' if it doesn't exist
folder_path = "data"  # Defining the folder path as 'data'
if not os.path.exists(folder_path):  # Checking if the folder does not exist
    os.makedirs(folder_path)  # Creating the folder if it doesn't exist
    print(f"Folder '{folder_path}' created.")  # Printing confirmation that the folder was created
else:  # If the folder already exists
    print(f"Folder '{folder_path}' already exists.")  # Printing confirmation that the folder already exists


Folder 'data' already exists.


In [7]:
# [Student: Mohamed] Simulating 3000 transactions 
import pandas as pd  # Import pandas for data manipulation
import random  # Import random for random sampling


item_pool = [  # List of possible items in the supermarket
    "Milk", "Bread", "Eggs", "Butter", "Cheese", "Apples", "Bananas", "Oranges",
    "Chicken", "Beef", "Pasta", "Rice", "Tomatoes", "Onions", "Potatoes",
    "Lettuce", "Cucumber", "Carrots", "Yogurt", "Juice", "Cereal", "Coffee",
    "Tea", "Sugar", "Salt", "Cooking Oil", "Flour", "Toilet Paper", "Soap", "Shampoo"
    # 30 different items in total
 ]


transactions = []  # Initializing an empty list to store transactions
for _ in range(3000):  # Looping to create 3000 transactions
    num_items = random.randint(2, 7)  # Randomly choosing number of items in transaction (between 2 and 7)
    transaction = random.sample(item_pool, num_items)  # Randomly selecting items for the transaction
    transactions.append(transaction)  # Adding the transaction to the list

# Save as CSV
df_transactions = pd.DataFrame({'Transaction': transactions})  # Creating a DataFrame from the transactions list
df_transactions.to_csv('data/supermarket_transactions.csv', index=False)  # Saving the DataFrame to a CSV file
df_transactions.head()  # Displaying the first few transactions

,Transaction
0,"[Apples, Potatoes, Soap]"
1,"[Flour, Yogurt, Cooking Oil, Toilet Paper, Soa..."
2,"[Tea, Toilet Paper, Beef, Bananas, Potatoes]"
3,"[Apples, Rice]"
4,"[Apples, Salt]"
